In [ ]:
import numpy as np 
import pandas as pd 
import os
import re

In [ ]:
path = os.path.join(os.getcwd(),"Data")
stockspath = os.path.join(path,"Stock")
corporateactionspath = os.path.join(path,"CorporateActions")
# print(stockspath)
# print(corporateactionspath)
# print(path)

In [ ]:
def stock_split(stock,start_date,end_date,r1,r2):
    """
    For an r1:r2 stock split, if y is the stock value before the split,
    then the value of the stock will be y*(r1/r2),
    for the data between the given dates.

    Parameters
    ----------

    stock : dataframe

    start_date : datetime
    
    end_date : datetime

    r1 : integer

    r2 : integer

    Returns
    -------

    stock : dataframe
        updated dataframe after splitting
    """
    
    specific_dates = stock[stock.Date.between(end_date,start_date)]
    for index,row in specific_dates.iterrows():
        specific_dates.loc[index,"Open Price"] = specific_dates.loc[index,"Open Price"] * (r1/r2)
        specific_dates.loc[index,"Low Price"] = specific_dates.loc[index,"Low Price"] * (r1/r2)
        specific_dates.loc[index,"High Price"] = specific_dates.loc[index,"High Price"] * (r1/r2)
        specific_dates.loc[index,"Close Price"] = specific_dates.loc[index,"Close Price"] * (r1/r2)
        specific_dates.loc[index,"WAP"] = specific_dates.loc[index,"WAP"] * (r1/r2)
        stock.loc[index] = specific_dates.loc[index]
    return stock

In [ ]:
def bonus_issue(stock,start_date,end_date,r1,r2):
    """
    For an r1:r2 bonus shares,
    if y is the stock value before the bonus share issue,
    then the value of the stock will be y*(r2/(r1+r2)),
    for the data between the given dates.

    Parameters
    ----------

    stock : dataframe

    start_date : datetime
    
    end_date : datetime

    r1 : integer

    r2 : integer

    Returns
    -------

    stock : dataframe
        updated dataframe after bonus
    """

    specific_dates = stock[stock.Date.between(end_date,start_date)]
    for index,row in specific_dates.iterrows():
        specific_dates.loc[index,"Open Price"] = specific_dates.loc[index,"Open Price"] * (r2/(r1+r2))
        specific_dates.loc[index,"Low Price"] = specific_dates.loc[index,"Low Price"] * (r2/(r1+r2))
        specific_dates.loc[index,"High Price"] = specific_dates.loc[index,"High Price"] * (r2/(r1+r2))
        specific_dates.loc[index,"Close Price"] = specific_dates.loc[index,"Close Price"] * (r2/(r1+r2))
        specific_dates.loc[index,"WAP"] = specific_dates.loc[index,"WAP"] * (r2/(r1+r2))
        stock.loc[index] = specific_dates.loc[index]
    return stock

In [ ]:
def apply_corporate_actions(stock,corporate):

    """
    Applies stock split and bonus on the given stock dataset.

    creates bonus dataframe and invoke bonus_issue method.

    creates split dataframe and invoke stock_split method.

    Parameters
    ----------

    stock : dataframe
    
    corporate : dataframe

    Returns
    -------

    stock : dataframe
        updated dataframe after stock split and bonus.

    Methods
    -------
    stock_split : 

    bonus_issue :

    """

    stock["Date"] = pd.to_datetime(stock["Date"])
    corporate["Ex Date"] = pd.to_datetime(corporate["Ex Date"],errors='coerce')
    # corporate["BC Start Date"] = pd.to_datetime(corporate["BC Start Date"],errors='coerce')
    # corporate[" BC End Date\t"] = pd.to_datetime(corporate[" BC End Date\t"],errors='coerce')
    # corporate["ND Start Date"] = pd.to_datetime(corporate["ND Start Date"],errors='coerce')
    # corporate["ND End Date"] = pd.to_datetime(corporate["ND End Date"],errors='coerce')
    
    bonus_df = corporate[corporate['Purpose'].str.contains("Bonus")]
    for index,row in bonus_df.iterrows():
        start_date = bonus_df.loc[index,"Ex Date"]
        ratio = bonus_df.loc[index,"Purpose"]
        r1,r2 = re.findall(r"\d+",ratio)
        r1,r2 = int(r1),int(r2)
        end_date = stock.tail(1)["Date"].values[0]
        stock = bonus_issue(stock,start_date,end_date,r1,r2)

    stock_split_df = corporate[corporate['Purpose'].str.contains("Stock")]
    for index,row in stock_split_df.iterrows():
        start_date = stock_split_df.loc[index,"Ex Date"]
        ratio = stock_split_df.loc[index,"Purpose"]
        r1,r2 = re.findall(r"\d+",ratio)
        r1,r2 = int(r1),int(r2)
        end_date = stock.tail(1)["Date"].values[0]
        stock = stock_split(stock,start_date,end_date,r1,r2)
    return stock